In [1]:
import torch
from torch.utils.data import DataLoader
from dataset.dataset import get_cdiscount_dataset
from model.model import assemble_model
from trainer.trainer import get_trainer

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# redirect print to file
# import sys
# sys.stdout = open("PyTorch-resnet34-log.txt", "w")

In [ ]:
# parameters
config = {
    'train_batch_size': 48, 'val_batch_size': 48,
    'arch': 'resnet152',
    'optimizer': 'Adam', 'learning_rate': 5e-6, 'decay_lr_freq': 3e5, 'weight_decay': 5e-4,
    'resume': None,
    'start_epoch': 0, 'epochs': 10,
    'print_freq': 10, 'validate_freq': 3e4, 'save_freq': 1e3,
    'best_val_prec1': 0
}

In [ ]:
import torchvision.models as models

# get dataset
print('getting dataset...')
train_dataset = get_cdiscount_dataset(offsets_csv="train_offsets.csv",
                                      images_csv="train_images.csv",
                                      bson_file_path="/mnt/data/cdiscount/train.bson",
                                      with_label=True,
                                      resize=224)
val_dataset = get_cdiscount_dataset(offsets_csv="train_offsets.csv",
                                    images_csv="val_images.csv",
                                    bson_file_path="/mnt/data/cdiscount/train.bson",
                                    with_label=True,
                                    resize=224)

# get data loader
print('getting data loader...')
train_dataloader = DataLoader(train_dataset, batch_size=config['train_batch_size'], shuffle=True, num_workers=6)
val_dataloader = DataLoader(val_dataset, batch_size=config['val_batch_size'], shuffle=True, num_workers=6)

# define model
print("=> using pre-trained model '{}'".format(config['arch']))
model = models.__dict__[config['arch']](pretrained=True)
model = assemble_model(model, -1, 2048, 5270)
model = torch.nn.DataParallel(model).cuda()

# define loss function (criterion) and optimizer
criterion = torch.nn.CrossEntropyLoss().cuda()

# get trainer
Trainer = get_trainer(train_dataloader, val_dataloader, model, criterion, config)

# Run!
Trainer.run()

getting dataset...
getting data loader...
=> using pre-trained model 'resnet152'
start training
Epoch: [0][0/206270]	Time 23.269 (23.269)	Data 3.325 (3.325)	Loss 8.6347 (8.6347)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
Epoch: [0][10/206270]	Time 2.559 (4.451)	Data 0.007 (0.310)	Loss 8.6234 (8.6162)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
Epoch: [0][20/206270]	Time 2.557 (3.543)	Data 0.009 (0.167)	Loss 8.6363 (8.6019)	Prec@1 0.000 (0.000)	Prec@5 2.083 (0.397)
Epoch: [0][30/206270]	Time 2.564 (3.226)	Data 0.009 (0.116)	Loss 8.5990 (8.5756)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.336)
Epoch: [0][40/206270]	Time 2.549 (3.063)	Data 0.008 (0.090)	Loss 8.4095 (8.5665)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.254)
Epoch: [0][50/206270]	Time 2.558 (2.963)	Data 0.018 (0.074)	Loss 8.5124 (8.5533)	Prec@1 0.000 (0.000)	Prec@5 4.167 (0.408)
Epoch: [0][60/206270]	Time 2.562 (2.898)	Data 0.008 (0.064)	Loss 8.4454 (8.5297)	Prec@1 0.000 (0.102)	Prec@5 0.000 (0.444)
Epoch: [0][70/206270]	Time 2.546 (2.851)	D